# 🎯 Causal Forest Preprocessing for Customer Screen Preference Analysis

This script performs data preprocessing to prepare for causal inference using the **Causal Forest DML** method from the `econml` library. The goal is to estimate the causal effect of screen type (Large vs. Small) on user "Level" performance using various customer features.

---

## 🔢 Dataset Overview

- 📄 File: `RSR Largescreen_Smallscreen customer.xlsx`
- 🔍 Features:
  - `Feature T1`, `Feature M1`, `Feature C1`: Numeric covariates
  - `Screen`: Treatment variable (categorical: "L" = large screen)
  - `Level`: Outcome variable

---

## ⚙️ Key Processing Steps

### 1. Load and Clean Data

- Import data from Excel
- Remove whitespace from column names
- Fill missing values with column means

### 2. Define Variables

- **X**: Feature matrix containing T1, M1, C1
- **Y**: Target variable (`Level`)
- **T**: Binary treatment variable (1 = Large Screen, 0 = Small Screen)

### 3. Standardization (optional)

- Log transform (commented out in this version)
- Standardize values using `StandardScaler`

### 4. Visualization

- Plot histogram distributions of each feature
- Display feature correlation matrix using seaborn heatmap

---

## 📈 Output

- 📊 Feature summary statistics (before and after processing)
- 🧱 Feature distributions (histograms)
- 🔗 Feature correlation heatmap

---

## 🚀 Next Step

After this preprocessing, the prepared variables `X`, `Y`, and `T` are ready to be passed into the `CausalForestDML` model for estimating individual treatment effects.

---

> **Note**: This script uses `econml`, `scikit-learn`, `seaborn`, and `matplotlib` for preprocessing and visualization. Be sure to install the required packages before running.



In [ ]:
import numpy as np
import pandas as pd
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Excel data
file_path = "RSR Largescreen_Smallscreen customer.xlsx"
data = pd.read_excel(file_path)

# 2. Check and handle missing values
if data.isnull().sum().sum() > 0:
    print("Missing values detected. Handling missing values...")
    data.fillna(data.mean(), inplace=True)

# Strip any extra whitespace from column names
data.columns = data.columns.str.strip()

# 3. Display original feature statistics
print("\n--- Original Feature Statistics ---")
print(data[['Feature T1', 'Feature M1', 'Feature C1']].describe())

# (Optional) Apply log transformation to reduce skewness
# data['Feature T1'] = np.log1p(data['Feature T1'])
# data['Feature M1'] = np.log1p(data['Feature M1'])
# data['Feature C1'] = np.log1p(data['Feature C1'])

# Standardize all features
scaler = StandardScaler()
X = scaler.fit_transform(data[['Feature T1', 'Feature M1', 'Feature C1']])

# Prepare features and labels
X = data[['Feature T1', 'Feature M1', 'Feature C1']]
Y = data['Level'].values
T = data['Screen'].apply(lambda x: 1 if x == 'L' else 0).values  # L = 1 (Large screen), others = 0

# Ensure treatment variable is integer
T = T.astype(int)

# Display statistics after processing
print("\n--- Processed Feature Statistics ---")
print(pd.DataFrame(X, columns=['Feature T1', 'Feature M1', 'Feature C1']).describe())

# Plot histograms of processed features
pd.DataFrame(X, columns=['Feature T1', 'Feature M1', 'Feature C1']).hist(bins=30, figsize=(10, 6))
plt.suptitle("Processed Feature Distributions")
plt.show()

# Check feature correlation
print("\n--- Feature Correlation Matrix ---")
correlation_matrix = data[['Feature T1', 'Feature M1', 'Feature C1']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()


# 🧪 Stratified Train-Test Split for Causal Inference

This script performs a **stratified sampling split** of the dataset for causal inference, ensuring that the training and test sets maintain the same distribution across treatment groups and outcome levels.

---

## 🎯 Purpose

When estimating **Individual Treatment Effects (ITE)** using methods like Causal Forests, it's important that:

- Treatment assignment (`Screen`)
- Outcome levels (`Level`)

...are evenly distributed between training and testing datasets. This ensures that the model learns from a balanced and representative dataset.

---

## ⚙️ Key Steps

### 1. Clean and Format Target Variable

- Convert `Level` to integers
- Remove rows with unexpected levels (only keep 1, 2, or 3)

### 2. Create Stratification Variable

- Combine `Screen` and `Level` into a composite variable:  
  e.g., `L_1`, `S_3`, etc.
- Used for stratified sampling

### 3. Perform Stratified Split

- Use `train_test_split` with the composite `Stratify` column
- Split into training (80%) and testing (20%) sets

### 4. Validate the Split

- Print shape of split datasets
- Count stratification label distribution in each subset

---

## 📊 Output

- Balanced train/test splits across treatment & outcome strata
- Console output showing distribution consistency

---

## 🧠 Why This Matters

Stratified sampling is crucial in **causal machine learning** to ensure valid and unbiased comparisons between groups. If the treatment or outcome distribution is skewed post-split, it may introduce confounding or reduce generalizability.



In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Round and convert 'Level' to integer (e.g., in case of floating point noise)
data['Level'] = data['Level'].round().astype(int)

# Filter out invalid values (keep only Levels 1, 2, 3)
data = data[data['Level'].isin([1, 2, 3])]

# Create a new stratification variable combining treatment and outcome
data['Stratify'] = data['Screen'] + "_" + data['Level'].astype(str)

stratify_var = data['Stratify']

# Print unique values and counts for stratification
print("Unique values in Stratify column:", data['Stratify'].unique())
print("Stratify value counts:\n", data['Stratify'].value_counts())

# Perform stratified train-test split to preserve treatment and outcome balance
X_train, X_test, Y_train, Y_test, T_train, T_test, stratify_train, stratify_test = train_test_split(
    X, Y, T, stratify_var,
    test_size=0.2,
    random_state=42,
    stratify=stratify_var
)

# Check the shape of the split datasets
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("T_train shape:", T_train.shape)
print("Stratify_train shape:", len(stratify_train))

# Display stratification counts for training and testing sets
train_counts = pd.Series(stratify_train).value_counts()
test_counts = pd.Series(stratify_test).value_counts()

print("\nTraining Set Stratify Counts:\n", train_counts)
print("\nTest Set Stratify Counts:\n", test_counts)


# 🔍 One-Hot Encoding for Stratified Covariates

This script applies one-hot encoding to the stratification variable for use in advanced causal inference models (e.g., `CausalForestDML` from the `econml` package).

---

## 🎯 Purpose

Stratification variables often need to be converted into numeric form when used as control variables or covariates. Here, **one-hot encoding** ensures categorical stratification levels (like `"L_1"`, `"S_3"`) are usable in models that require numerical input.

---

## ⚙️ Key Steps

### 1. One-Hot Encoding

- Encode the composite stratification variable (`Screen_Level`)
- Use `OneHotEncoder` from `sklearn`
- `sparse_output=False` ensures dense matrix for model compatibility

### 2. Array Shape Validation

- Print shapes of encoded arrays
- Ensure number of samples match across:
  - `X_train` (features)
  - `Y_train` (target)
  - `stratify_train_encoded` (controls)

### 3. Type Checks (Debug)

- Confirm data structure integrity

---

## ✅ Output

- Encoded stratification matrices (train & test)
- Console output with shape info
- Assertion to verify row alignment

---

## 💡 Why It Matters

Causal inference models like **Double Machine Learning (DML)** require precise alignment of:

- Treatments
- Outcomes
- Covariates (controls)

Improper encoding can lead to **dimension mismatches** or **biased estimates**. One-hot encoding helps safely incorporate stratification into the causal framework.



In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# One-hot encode the stratification variable
encoder = OneHotEncoder(sparse_output=False)

# Reshape the Series to 2D array for encoder input
stratify_train_encoded = encoder.fit_transform(stratify_train.values.reshape(-1, 1))
stratify_test_encoded = encoder.transform(stratify_test.values.reshape(-1, 1))

# Print encoded shape for verification
print("stratify_train_encoded shape:", stratify_train_encoded.shape)
print("stratify_test_encoded shape:", stratify_test_encoded.shape)

# Print original training data shape
print("X_train shape:", X_train.shape)  # (n_samples, n_features)
print("Y_train shape:", Y_train.shape)  # (n_samples,)

# Sanity check: number of samples must match across arrays
assert X_train.shape[0] == Y_train.shape[0] == stratify_train_encoded.shape[0], "Inconsistent sample sizes!"

# Print object types for debugging
print(type(stratify_train))
print(type(stratify_train.values))


# 🌲 Causal Forest with Hyperparameter Optimization

This script builds a **Causal Forest model** using `econml`'s `CausalForestDML`, with a full pipeline including **data cleaning**, **hyperparameter tuning**, and **treatment effect estimation**.

---

## 🧠 Key Components

- **Outcome Model (`model_y`)**: `RandomForestRegressor`, optimized using `GridSearchCV` with `r2` as the scoring metric.
- **Treatment Model (`model_t`)**: `RandomForestClassifier`, optimized with `f1_weighted` scoring.
- **Causal Estimation**: Estimates **heterogeneous treatment effects (HTE)** and **average treatment effect (ATE)** using the Causal Forest algorithm.

---

## ⚙️ Workflow

### Step 1: Preprocessing
- Split dataset into training and test sets.
- Replace missing or invalid values using `np.nan_to_num`.

### Step 2: Model Optimization
- Use `GridSearchCV` to tune hyperparameters for both the outcome and treatment models.
- Models are selected based on cross-validated performance.

### Step 3: Train Causal Forest
- Initialize `CausalForestDML` with optimized sub-models.
- Fit the model using the training data.

### Step 4: Evaluate
- Predict **individual treatment effects** using `.effect()`.
- Estimate **Average Treatment Effect (ATE)** using `.ate()`.
- Compute **confidence intervals** using `.ate_interval()`.

### Step 5: Visualization
- Histogram of estimated treatment effects.
- Summary statistics and cross-validated scores.

---

## 📈 Output

- Optimized parameters for `model_y` and `model_t`
- Cross-validated F1 scores for treatment model
- ATE estimate and confidence interval
- Treatment effect distribution plot

---

## 🔍 Notes

- `discrete_treatment=True` is required since treatment is binary.
- A high number of trees (`n_estimators=1000`) ensures more stable effect estimation.



In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from econml.dml import CausalForestDML
from sklearn.metrics import accuracy_score, f1_score, classification_report, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Split data into training and test sets
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

# Step 2: Clean the data (handle NaNs and negative values if needed)
Y_train = np.nan_to_num(Y_train)
T_train = np.nan_to_num(T_train)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# Step 3: Define hyperparameter grid for model tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Step 4: Grid search for outcome model (Y)
grid_search_y = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='r2')
grid_search_y.fit(X_train, Y_train)

# Step 5: Grid search for treatment model (T)
grid_search_t = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_weighted')
grid_search_t.fit(X_train, T_train)

# Step 6: Use best estimators
optimized_model_y = grid_search_y.best_estimator_
optimized_model_t = grid_search_t.best_estimator_

print("Optimized model_y parameters:", grid_search_y.best_params_)
print("Optimized model_t parameters:", grid_search_t.best_params_)

# Step 7: Initialize causal forest using optimized models
causal_forest = CausalForestDML(
    model_y=optimized_model_y,
    model_t=optimized_model_t,
    discrete_treatment=True,
    n_estimators=1000,
    random_state=42
)

# Step 8: Evaluate treatment model using cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(causal_forest.model_t, X_train, T_train, cv=kf, scoring='f1_weighted')
print("Cross-Validation F1 Scores for Treatment Model:", cross_val_scores)
print("Mean F1 Score:", np.mean(cross_val_scores))

# Step 9: Fit causal forest
try:
    causal_forest.fit(Y_train, T_train, X=X_train)
    print("Causal forest model fitted successfully.")
except Exception as e:
    print(f"Error during fitting: {e}")

# Step 10: Estimate treatment effects
try:
    treatment_effects = causal_forest.effect(X_test)
    print("Treatment Effects (first 10):", treatment_effects[:10])
except Exception as e:
    print(f"Error predicting treatment effects: {e}")

# Step 11: Estimate average treatment effect (ATE)
try:
    ate = causal_forest.ate(X_test)
    print("Average Treatment Effect (ATE):", ate)
except Exception as e:
    print(f"Error computing ATE: {e}")

# Step 12: Plot treatment effect distribution
try:
    plt.hist(treatment_effects, bins=30, edgecolor='k')
    plt.title("Distribution of Estimated Treatment Effects")
    plt.xlabel("Treatment Effect")
    plt.ylabel("Frequency")
    plt.show()
except Exception as e:
    print(f"Error in plotting treatment effect distribution: {e}")

# Step 13: Compute confidence interval for ATE
try:
    lb, ub = causal_forest.ate_interval(X_test)
    print(f"ATE Confidence Interval: [{lb}, {ub}]")
except Exception as e:
    print(f"Error computing ATE confidence interval: {e}")


# 🎯 Individual Treatment Effect (ITE) Analysis with Causal Forest

This workflow focuses on estimating and analyzing **individual treatment effects (ITE)** using a fitted `CausalForestDML` model, along with **grouped comparisons**, **feature importance extraction**, and **visualizations**.

---

## 🧪 Core Workflow

### Step 1: Estimate ITE
- Use `.effect(X)` to compute the personalized treatment effect for each observation.

### Step 2: Grouping by Device Type and Engagement Level
- Create a summary DataFrame with:
  - `ITE`: predicted individual treatment effect
  - `DeviceType`: treatment group (e.g., large screen vs. small screen)
  - `EngagementLevel`: behavioral engagement (1, 2, 3)

- Compute average treatment effect within each subgroup:
```python
grouped_df = ite_df.groupby(['DeviceType', 'EngagementLevel'])['ITE'].mean()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Predict Individual Treatment Effects (ITE)
ite = causal_forest.effect(X)

# Extract device type column (assumed to represent treatment assignment)
device_type = data['Screen']  # e.g., 'L' for large, 'S' for small

# Ensure engagement level column is valid
data['Level'] = data['Level'].round().astype(int)  # Convert to integer
data = data[data['Level'].isin([1, 2, 3])]         # Keep only valid levels

# Validate data cleaning
if set(data['Level'].unique()) <= {1, 2, 3}:
    print("Engagement level cleaned successfully: only 1, 2, 3 present.")
else:
    raise ValueError("Invalid values remain in Level column.")

# Create DataFrame for ITE analysis
ite_df = pd.DataFrame({
    'ITE': ite,
    'DeviceType': data['Screen'],
    'EngagementLevel': data['Level']
})

# Compute group-wise average ITE
group_effects = ite_df.groupby(['DeviceType', 'EngagementLevel'])['ITE'].mean().reset_index()
print("Grouped Effects:")
print(group_effects)

# Extract feature importances
feature_importances = causal_forest.feature_importances()
behavior_variables = ['T1', 'M1', 'C1']  # Assumed behavioral features

importances_df = pd.DataFrame({
    'Feature': behavior_variables,
    'Importance': feature_importances
})
print("Feature Importance:")
print(importances_df)

# Visualize ITE distribution by device type and engagement level
plt.figure(figsize=(10, 6))
sns.boxplot(data=ite_df, x='DeviceType', y='ITE', hue='EngagementLevel')
plt.title('ITE Distribution by Device Type and Engagement Level')
plt.xlabel('Device Type')
plt.ylabel('Individual Treatment Effect (ITE)')
plt.legend(title='Engagement Level')
plt.show()

# Visualize feature importance of behavioral variables
plt.figure(figsize=(8, 6))
sns.barplot(data=importances_df, x='Feature', y='Importance')
plt.title('Feature Importance of Behavioral Variables')
plt.xlabel('Behavioral Variable')
plt.ylabel('Importance')
plt.show()

# Optional: Fit causal forest using built-in cross-validation
causal_forest = CausalForestDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    discrete_treatment=True,
    cv=5,
    random_state=42
)

# Fit model
causal_forest.fit(Y_train, T_train, X=X_train)
print("Causal forest fitted successfully with cross-validation!")
